# Regular Expression composer

本程式可以在 [Google Colab 上直接執行](https://colab.research.google.com/github/hcchengithub/RegEx-composer/blob/main/RegEx%20composer.ipynb)，不需要任何安裝。

Hubble2 搜尋近似 bugs 的結果輸出到一個 text file, 善用 Notepad++ 可從中提煉出精準的結果。 本工具用來輔助產生需要用到的 Notepad++ regular expression。 

Notepad++ 能任意給 words 塗顏色。理想中我們希望依顏色組成搜尋條件：相同顏色取 or 邏輯，任一即可；不同顏色取 and 邏輯，要全部到齊。例如： 

    1. bluetooth, bt 都塗成「藍色」。 
    2. Windows 10, win10, windows10, window10 都塗成「綠色」。
    3. blue screen, bsod, fatal, exception 都塗成「紅色」。

Notepad++ (請 update 到最新版，已知 v7.9.5 之前的版本不太好) 的 find 功能有提供 regular expression 能達到這個效果。以上三個 pattern 可用本程式轉換成 regular expression 供 Notepad++ 使用，以達到精準搜尋的目的。

<img src="https://github.com/hcchengithub/RegEx-composer/blob/main/Screenshot%202021-10-15%20094215.jpg?raw=1">

Notepad++ 有個 PythonScript plugin. 等將來寫好了就只要塗顏色，然後執行 PythonScript 即可，方便點。


# Usage:

直接在下面 source code 裡面修改 patterns 然後 Ctrl+Enter 執行，產生的 RegEx 在下面。請 copy-paste 到 Notepad++ 使用。

In [17]:
# patterns from the notepad++ style token colors 
# (?=.*?\s(windows?.?10|win.?10)\s)(?=.*?\s(blue.?screen|BSOD)\s).*
patterns = []
patterns.append("blue.?tooth|bt") # .? 有沒有空格或任何 character 都接受。 '|' 就是 OR. 
patterns.append("windows?.?10|win10") # s? 有沒有 s 都接受， .? 解釋如上
patterns.append("blue.?screen|bsod|fatal|exception")
patterns.append("")
patterns.append("") # 自行增添

# 全部有效 OR patterns
patterns = [p.strip() for p in patterns if p.strip()] # remove empty items
for i in range(len(patterns)):
    s = patterns[i]
    patterns[i] = r"(?=^.*?\s(%s)\s)" % s # https://community.notepad-plus-plus.org/topic/22031/first-f3-ok-f3-again-selects-the-entire-doc

# 各種 OR patterns 全部要到齊，都 AND 起來。
regex = ""
for i in patterns: 
    regex += i

# 輸出結果
# print('len(regex) =',len(regex), '<--- 不要超過 2046 即可\n')
assert len(regex) <= 2046, "Notepad++ regular expression length <= 2046 limitation!"
print("Regular Expression:")
print("  %s.*" % regex)

Regular Expression:
  (?=^.*?\s(blue.?tooth|bt)\s)(?=^.*?\s(windows?.?10|win10)\s)(?=^.*?\s(blue.?screen|bsod|fatal|exception)\s).*


# The End

陳厚成 H.C. Chen<br>
緯創資通 Wistron Corporation<br>
新北市22181汐止區新台五路一段88號21樓<br>
21F., No.88, Sec.1, Xintai 5th Rd., Xizhi Dist.,<br>
New Taipei City 22181, Taiwan<br>
eMail: h.c._chen@wistron.com<br>
Phone: +886-2-6612-27887, Cell: +886-922-417-555<br>
Line: hcchen5600, WeChat: hcchen5600



# 筆記
得到 [Alan Kilborn](https://community.notepad-plus-plus.org/topic/21973/regular-expression-size-limited-2048-chars-how-to-make-it-longer/9?_=1634781956921) 的協助之前的舊版
```
# patterns from the notepad++ style token colors 

patterns = []
patterns.append("blue.?tooth|bt") # .? 有沒有空格都接受
patterns.append("windows?.?10|win10") # s? 有沒有 s 都接受
patterns.append("blue.?screen|bsod|fatal|exception")
patterns.append("")
patterns.append("") # 自行增添

from itertools import permutations  # permutations 數學「排列組合」中的「排列」

# 所有 patterns 的排列
patterns = [p.strip() for p in patterns if p.strip()] # remove empty items
for i in range(len(patterns)):
    s = patterns[i]
    patterns[i] = '(' + s + ')' if s.find('|') != -1 else s # 給有 | 的用括號括起來
permus = [p for p in permutations(patterns)] 

# RegEx 模板隨 input patterns 個數而定，自動伸縮。
template = "\s"
for i in range(len(patterns)): 
    # 萬一 RegEx 太長超過 Notepad++ 的限制 2048 characters 則這裡犧牲一點還可以縮短
    # template += r"%s.*\s" # 注意，尾巴要剪掉的長度要連動 -4 ----------.
    template += r"%s(\s|\s.*\s)" # 尾巴長度 -11                       |
template = template[:-11]+"\\s" # <----------------------------------'

# 各種排列組合的聯集
regex = ""
for i in range(len(permus)): 
    regex += "(%s)" % template % tuple([p for p in permus[i]]) + '|'
regex = regex[:-1]

# 輸出結果
print('len(regex) =',len(regex), '<---控制不要超過 2048 即可\n')
assert len(regex) <= 2048, "Notepad++ regular expression length limitation! 2048 不太夠，很容易超過！"
print(regex)
```